<a href="https://colab.research.google.com/github/ActiveStacks/AI_with_PyTorch/blob/main/ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dogs vs Cats Image Classification withut Augmentation


In [33]:
from __future__ import absolute_import, division, print_function

In [34]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [35]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# tf.compat.v1.loggng
# # tf.logging.set_verbosity(tf.logging.ERROR)


In [36]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'


In [37]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

****

In [38]:
zip_dir_base = os.path.dirname(zip_dir)
!find $zip_dir_base -type d -print

/root/.keras/datasets
/root/.keras/datasets/cats_and_dogs_filtered
/root/.keras/datasets/cats_and_dogs_filtered/train
/root/.keras/datasets/cats_and_dogs_filtered/train/cats
/root/.keras/datasets/cats_and_dogs_filtered/train/dogs
/root/.keras/datasets/cats_and_dogs_filtered/validation
/root/.keras/datasets/cats_and_dogs_filtered/validation/cats
/root/.keras/datasets/cats_and_dogs_filtered/validation/dogs


In [39]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validaion_dogs_dir = os.path.join(validation_dir, 'dogs')

**Understanding Our Data**

In [40]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validaion_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

print('total training cats images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("__")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training cats images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
__
Total training images: 2000
Total validation images: 1000


**SETTING THE MODEL PARAMETERS**
Batch size is the number of training examples to process before updating our models variables


In [41]:
BATCH_SIZE = 100
IMG_SHAPE = 150

**DATA PREPARATION**

In [42]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [43]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_SHAPE, IMG_SHAPE),
                                                           class_mode='binary')

Found 2000 images belonging to 2 classes.


In [44]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(IMG_SHAPE, IMG_SHAPE),
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.


In [45]:
# VISUALIZING TRAINING IMAGES
sample_training_images, _ =next(train_data_gen)

**PLOTTING IMAGES**

In [46]:
def plotImages(images_arr):
  fig, axes = plt.subplots(1, 5, figsize(20,20))
  axes = axes.flatten()
  for img, ax in zip( images_arr, axes):
    ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [47]:
# plotImages(sample_training_images[:5])

**MODEL CREATION**

In [48]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation ='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

**COMPILE THE MODEL**

In [49]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

**MODEL SUMMARY**

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 17, 17, 128)      

**TRAIN THE MODEL**

In [51]:
EPOCHS = 100
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs = EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

<ipython-input-51-a5d43c6ee11a>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/100
20/20 [==============================] - 13s 268ms/step - loss: 0.7859 - accuracy: 0.5280 - val_loss: 0.7011 - val_accuracy: 0.5000
Epoch 2/100
20/20 [==============================] - 6s 305ms/step - loss: 0.6925 - accuracy: 0.5145 - val_loss: 0.6819 - val_accuracy: 0.5200
Epoch 3/100
20/20 [==============================] - 5s 248ms/step - loss: 0.6692 - accuracy: 0.5755 - val_loss: 0.7652 - val_accuracy: 0.4990
Epoch 4/100
20/20 [==============================] - 6s 291ms/step - loss: 0.6717 - accuracy: 0.5885 - val_loss: 0.6203 - val_accuracy: 0.6530
Epoch 5/100
20/20 [==============================] - 5s 250ms/step - loss: 0.6508 - accuracy: 0.5905 - val_loss: 0.6388 - val_accuracy: 0.6470
Epoch 6/100
20/20 [==============================] - 6s 307ms/step - loss: 0.6080 - accuracy: 0.6785 - val_loss: 0.6099 - val_accuracy: 0.6770
Epoch 7/100
20/20 [==============================] - 7s 356ms/step - loss: 0.5621 - accuracy: 0.7185 - val_loss: 0.5800 - val_accuracy: 0.698

**VISUALIZING THE RESULTS**

In [52]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range[EPOCHS]

plt.figure(figsize=(8, 8)
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Traning Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation')
plt.save('./foo.png')
plt.show()


SyntaxError: '(' was never closed (<ipython-input-52-04fffd3d4de4>, line 9)